# Total points 8

# In  the homework
- We'll collect climate change related tweet from twitter using REST API
- Use tweets to populate Pandas DataFrames
- Perform some elementary pandas operation to check data sanity


**Please read**

- https://developer.twitter.com/en/developer-terms/more-on-restricted-use-cases


Before proceeding with homework.

<font size="6" color='red'> Create new app at https://apps.twitter.com/app/new  (You may have to create twitter account before this) </font>





[twitter api reference index](https://developer.twitter.com/en/docs/api-reference-index)


# POST oauth2/token
Allows a registered application to obtain an OAuth 2 Bearer Token, which can be used to make API requests on an application's own behalf, without a user context.
https://developer.twitter.com/en/docs/basics/authentication/api-reference/token


Once you have create app, click on app details and keys and token tab

<font size="6" color='red'> write API key value for client_key and API secret key value for client_secret in the next cell</font>

In [ ]:
client_key = #write your API key
client_secret =  #write your API secret key 

In [ ]:
import requests
import base64

key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [4]:
base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}



Let's post to authentication endpoint and get the token to be used in subsequent REST API call

In [5]:
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [6]:
json_data =  response.json()
json_data

{'token_type': 'bearer',
 'access_token': 'AAAAAAAAAAAAAAAAAAAAAB%2FRYgEAAAAAX6%2BAKiphwwLFPl9cLKpSAiK%2FqB8%3DGm5aXL6goP919Zzw2zr1vRgZU9LKIknUYXTlVxvBK7AtAVklxQ'}

In [7]:
access_token = json_data['access_token']

# Let's do some search on climate change
This is how we got search endpoint

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

In [12]:
max_tweets=100
tw_limit=100# we can get only 100 tweet per call with standard search api
tweets = []
users =[]

In [18]:
search_headers = {'Authorization': 'Bearer {}'.format(access_token)    
}

parameters = { 'query': 'climate change',
                'max_results': 100,
                'expansions' :'author_id,referenced_tweets.id',
                'tweet.fields' : 'created_at,entities,source,in_reply_to_user_id,geo,lang,public_metrics,reply_settings,referenced_tweets,conversation_id',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'}

search_url = base_url+'2/tweets/search/recent'

response = requests.get(search_url, headers=search_headers, params=parameters)

In [19]:
climate_tweets = response.json()


In [41]:
len(climate_tweets['data'])

100

In [42]:
tweets +=climate_tweets['data']


In [43]:

 ids = [tw['id'] for tw in climate_tweets['data']]
    

In [44]:
min_id = min(ids)

In [45]:
tw_ids = []
tw_ids+=ids

As per twitter api documentation we can use

until_id:Returns results with an ID less than (that is, older than) or equal to the specified ID.

to get older tweets

# Q1 (.5 points) Modify only *parameters* dictionary in the following code so that subsequent call to climate change search returns older tweets with id less than minimum of previous tweets ids.

In [46]:
 parameters = { 'query': 'climate change',
                'max_results': tw_limit,
                'expansions' :'author_id,referenced_tweets.id',
                'tweet.fields' : 'created_at,entities,source,in_reply_to_user_id,geo,lang,public_metrics,reply_settings,referenced_tweets,conversation_id',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'
                 }

for  i in range(max_tweets//tw_limit -1):
    print("searching tweets with id  < {}".format(min_id))
    search_url = base_url+'2/tweets/search/recent'
    response = requests.get(search_url, headers=search_headers, params=parameters)
    climate_tweets = response.json()
    ids = [tw['id'] for tw in climate_tweets['statuses']]
    tw_ids+=ids
    print(i, len(climate_tweets['data']))
    min_id = min(ids)
    tweets +=climate_tweets['data']
    users.extend(climate_tweets['includes']['users'])
    if 'next_token' in response.json()['meta']:
					parameters['next_token'] = response.json()['meta']['next_token']

In [47]:
print('Total tweets retrieved= {}'.format(len(tweets)))

Total tweets retrieved= 300


# Let's print some tweets

In [48]:
import pprint #pretty-print” arbitrary Python data structures. Tweets object has hierarchical struture
max_tweet_to_print=4

In [28]:
pprint.pprint(tweets[0:max_tweet_to_print])

[{'author_id': '1315775267831382018',
  'conversation_id': '1526375358383017985',
  'created_at': '2022-05-17T23:23:01.000Z',
  'entities': {'mentions': [{'end': 16,
                             'id': '1497960029374201858',
                             'start': 0,
                             'username': 'GreenJDandWords'},
                            {'end': 31,
                             'id': '1004132506805854208',
                             'start': 17,
                             'username': 'TiemannAmelia'}]},
  'id': '1526704873995046912',
  'in_reply_to_user_id': '1315775267831382018',
  'lang': 'en',
  'public_metrics': {'like_count': 0,
                     'quote_count': 0,
                     'reply_count': 0,
                     'retweet_count': 0},
  'referenced_tweets': [{'id': '1526704632960909312', 'type': 'replied_to'}],
  'reply_settings': 'everyone',
  'source': 'Twitter Web App',
  'text': '@GreenJDandWords @TiemannAmelia For myself, and most of the nuclear 

Link to understand [tweet-object](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object)

Let'see some tweets

In [49]:
for status in tweets[0:max_tweet_to_print]:
    print(status['text'] + '\n')

@GreenJDandWords @TiemannAmelia For myself, and most of the nuclear advocates I know, climate change is the primary motivation!  2/2

A2  🌺Joy
       🌺 Students need to learn where and how they have food
       🌺 To learn about Climate Change students need to know what plants/trees need to thrive 
       🌺 Understanding Life Cycles 
#2PencilChat

RT @suzseddon: Why is Prince Charles saying if we don’t tackle climate change,we will have more pandemic’s. Wtf has pandemic’s got to do wi…

RT @Greg_T_NSW: @murpharoo @msmarto The Coalition have cut the Public Service over many years, demoralising staff and reducing services to…



# Pandas DataFrame for storing tweet feed

Create following date frame for storing tweets

- **climate_feed_df** DataFrame for storing tweet
- **user_df** DataFrame for storing user detail
- **retweeted_status_df** for storing original tweet

# Printing keys in tweets

In [56]:
print(tweets[1].keys())
print(len(tweets[1].keys()))


dict_keys(['id', 'entities', 'source', 'conversation_id', 'created_at', 'public_metrics', 'author_id', 'reply_settings', 'lang', 'text'])
10


In [57]:
import pandas as pd
import copy

# Generators for building DataFrames

# Q2 (2 point) Complete following   python generator *tweet_generator*. Generator yield modified tweet .
For each tweet in tweets
- Replace **author_id** attribute with **user_id**.
- Replace **referenced_tweets** attribute with **referenced_tweets_ids** and value(s) with **tweet['referenced_tweets']['id']** value(s).
- Keep other attributes in the tweet as such.

In [33]:
def tweet_generator(tweets):
    for tweet in tweets:
        modified_tweet= {}
        
        #Write your code here
        
        
        yield modified_tweet       
            
            

In [58]:
def tweet_generator(tweets):
    for tweet in tweets:
        try:
            modified_tweet = copy.deepcopy(tweet)
            modified_tweet['user_id'] = modified_tweet.pop('author_id')
            modified_tweet['referenced_tweets_ids'] = modified_tweet['referenced_tweets'][0]['id']
            modified_tweet['retweet_count'] = modified_tweet['public_metrics']['retweet_count']
            yield modified_tweet     
        except KeyError as e:
            continue

In [59]:
# Checking  generator output
list(tweet_generator(tweets[0:2]))

[{'id': '1526704873995046912',
  'source': 'Twitter Web App',
  'referenced_tweets': [{'type': 'replied_to', 'id': '1526704632960909312'}],
  'conversation_id': '1526375358383017985',
  'created_at': '2022-05-17T23:23:01.000Z',
  'public_metrics': {'retweet_count': 0,
   'reply_count': 0,
   'like_count': 0,
   'quote_count': 0},
  'entities': {'mentions': [{'start': 0,
     'end': 16,
     'username': 'GreenJDandWords',
     'id': '1497960029374201858'},
    {'start': 17,
     'end': 31,
     'username': 'TiemannAmelia',
     'id': '1004132506805854208'}]},
  'reply_settings': 'everyone',
  'lang': 'en',
  'in_reply_to_user_id': '1315775267831382018',
  'text': '@GreenJDandWords @TiemannAmelia For myself, and most of the nuclear advocates I know, climate change is the primary motivation!  2/2',
  'user_id': '1315775267831382018',
  'referenced_tweets_ids': '1526704632960909312',
  'retweet_count': 0}]

In [60]:
climate_feed_df = pd.DataFrame(tweet_generator(tweets))

In [61]:
climate_feed_df

,id,source,referenced_tweets,conversation_id,created_at,public_metrics,entities,reply_settings,lang,in_reply_to_user_id,text,user_id,referenced_tweets_ids,retweet_count
0,1526704873995046912,Twitter Web App,"[{'type': 'replied_to', 'id': '152670463296090...",1526375358383017985,2022-05-17T23:23:01.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'mentions': [{'start': 0, 'end': 16, 'usernam...",everyone,en,1315775267831382018,"@GreenJDandWords @TiemannAmelia For myself, an...",1315775267831382018,1526704632960909312,0
1,1526704868882370561,Twitter for Android,"[{'type': 'retweeted', 'id': '1526661121939329...",1526704868882370561,2022-05-17T23:23:00.000Z,"{'retweet_count': 120, 'reply_count': 0, 'like...","{'mentions': [{'start': 3, 'end': 13, 'usernam...",everyone,en,NaN,RT @suzseddon: Why is Prince Charles saying if...,1436492649473708070,1526661121939329026,120
2,1526704866542100480,Twitter for Android,"[{'type': 'retweeted', 'id': '1526396878828748...",1526704866542100480,2022-05-17T23:22:59.000Z,"{'retweet_count': 28, 'reply_count': 0, 'like_...","{'mentions': [{'start': 3, 'end': 14, 'usernam...",everyone,en,NaN,RT @Greg_T_NSW: @murpharoo @msmarto The Coalit...,25661191,1526396878828748800,28
3,1526704835365462016,Twitter for Android,"[{'type': 'retweeted', 'id': '1526283394526679...",1526704835365462016,2022-05-17T23:22:52.000Z,"{'retweet_count': 10, 'reply_count': 0, 'like_...","{'mentions': [{'start': 3, 'end': 14, 'usernam...",everyone,en,NaN,RT @Blue22Dave: The sky is falling! Sky is blu...,2849064914,1526283394526679040,10
4,1526704822941937664,Twitter for Android,"[{'type': 'retweeted', 'id': '1526704312860409...",1526704822941937664,2022-05-17T23:22:49.000Z,"{'retweet_count': 2, 'reply_count': 0, 'like_c...","{'mentions': [{'start': 3, 'end': 13, 'usernam...",everyone,en,NaN,RT @MrDenmore: The public cares most about cli...,442562435,1526704312860409856,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,1526704563306532864,Twitter Web App,"[{'type': 'retweeted', 'id': '1526492333973286...",1526704563306532864,2022-05-17T23:21:47.000Z,"{'retweet_count': 345, 'reply_count': 0, 'like...","{'hashtags': [{'start': 42, 'end': 49, 'tag': ...",everyone,en,NaN,"RT @vanbadham: Here’s a question for you, #aus...",1482269324337647617,1526492333973286913,345
264,1526704562303815680,Twitter for iPhone,"[{'type': 'retweeted', 'id': '1526600610165841...",1526704562303815680,2022-05-17T23:21:47.000Z,"{'retweet_count': 1761, 'reply_count': 0, 'lik...","{'mentions': [{'start': 3, 'end': 15, 'usernam...",everyone,en,NaN,RT @GavinNewsom: What a “pro-life” agenda ACTU...,1302639795714351104,1526600610165841921,1761
265,1526704556125585408,Twitter for Android,"[{'type': 'retweeted', 'id': '1526703477103149...",1526704556125585408,2022-05-17T23:21:45.000Z,"{'retweet_count': 2, 'reply_count': 0, 'like_c...","{'annotations': [{'start': 34, 'end': 89, 'pro...",everyone,en,NaN,RT @shanthropology: Today’s read: The Rise of ...,950085594478202880,1526703477103149056,2
266,1526704555064340480,Twitter for Android,"[{'type': 'retweeted', 'id': '1526704202902274...",1526704555064340480,2022-05-17T23:21:45.000Z,"{'retweet_count': 1, 'reply_count': 0, 'like_c...","{'mentions': [{'start': 3, 'end': 14, 'usernam...",everyone,en,NaN,RT @PerkasaJoB: Lack of organic content turns ...,133347754,1526704202902274048,1


# Q3(1 points) List datatype using climate_feed_df. Describe which attributes doesn't have correct data types. What should be correct datatype?

In [ ]:
# write code here datatype listing


# Write columns with wrong datatype and mention correct data type too.

*double click this cell to edit*


# Q4 (1.5 point) Complete following tweet_user_generator. Generator should yield only *user* value from a tweet

- As per the twitter API the user information is in the 'users' object within the 'includes' object of the response.

In [ ]:
def tweet_user_generator(tweets):
# write code here  


In [66]:
user_df = pd.DataFrame(tweet_user_generator(climate_tweets))

In [67]:
user_df

,0
0,data
1,includes
2,meta


# Q5(1 point) List datatypes of columns using user_df. Describe which attributes doesn't have correct data types. What should be correct datatype for each case?

In [ ]:
# write code here for datatype listing



# write columns with wrong datatype and mention correct data type too.

*double click this cell to edit*

# Q6 (1 point) Complete following retweeted_status_generator. Generator should yield dictionary of retweeted_status value from each tweet. If a tweet doesn't have this key return empty dictionary.

- As per the twitter API the user information is in the 'users' object within the 'includes' object of the response.

In [69]:
retweeted_status_df = climate_feed_df[['id', 'retweet_count']]

In [70]:
retweeted_status_df.head()

,id,retweet_count
0,1526704873995046912,0
1,1526704868882370561,120
2,1526704866542100480,28
3,1526704835365462016,10
4,1526704822941937664,2


# Q7 (1 point) Write code to find number of rows in retweeted_status_df that were not retweeted. print number of such rows.

In [ ]:
# Write code here
